## 手写数字识别案例

### 数据创建

MNIST基本上相当于机器学习的“Hello world”，每个进入该领域的人都需要好好学习该数据集的训练和测试。  

这是以个手写数字图像的数据集，是由0-9个10个数字构成的图像。里面训练图像有6万状，测试图像有1万张。我们就需要将让机器自己来对这10个数字进行分类：给一个图像，机器来告诉我们这个数字是几。

具体数据的导入如下：


In [5]:
import sys,os
# 将文件系统的搜索路径扩大到到当前文件的上一层
# 比如当前文件夹是：ch03,
# 加入了os.pardir,意味着将目录树向上遍历一级（例如，“ ..”）
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
# 导入数据
# 第 1 个参数 normalize 设置是否将输入图像正规化为 0.0～1.0 的值
# 第 2 个参数 flatten 设置是否展开输入图像（变成一维数组）。
# 第 3 个参数one_hot_label 设置是否将标签保存为 one-hot 表示
(x_train,y_train),(x_test,y_test) = load_mnist(flatten=True,
                                              normalize=False)
# 28*28的像素点：784
# 60000个训练数据
print(x_train.shape)
print(y_train.shape)
# 10000个测试数据
print(x_test.shape)
print(y_test.shape)

..
(60000, 784)
(60000,)
(10000, 784)
(10000,)


上面我们加载进来MNIST数据集，我们也查看该数据集的一些基本的东西，现在来看看具体加载图像显示。
![](imgs/18.jpg)



In [3]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
# 图像的显示使用PIL（Python Image Library）模块
from PIL import Image

def img_show(img):
    #  NumPy 数组的图像数据转换为 PIL 用的数据对象
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True,
normalize=False)
img = x_train[10]
label = y_train[10]
print(label)

print(img.shape)
# reshape() 方法的参数指定期望的形状，
img = img.reshape(28,28)
print(img.shape)

img_show(img)

3
(784,)
(28, 28)


### 利用神经网络进行推理

这里我们利用上面小节介绍的内容来实践这个手写数字识别案例。首先从输入层开始，我们这里设计两个2个隐藏层
- 输入层：784个输入，因为每个像素都可以作为一个输入
- 第一个隐藏层： 50个神经元
- 第二个隐藏层： 100个神经元
- 输出层：10个输出，因为我们就需要对0-9个数字进行分类

这里作者给出了一组参数的数据，该数据保存在pkl文件中。coding如下


In [2]:
def sigmoid(x):
    return 1.0/(np.exp(-x)+1)

def softmax(a):
    c= np.max(a)
    b = a - c # 防止溢出
    exp_a = np.exp(b)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y

In [5]:
import sys,os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
import numpy as np
import pickle
# 获得需要测试的数据
def get_data():
    (x_train, y_train), (x_test, y_test) = load_mnist(flatten=True,
normalize=True, one_hot_label=False)
    return x_test,y_test

# 获得网络的参数
def init_network():
    with open("sample_weight.pkl","rb") as f:
        network = pickle.load(f)
        
    return network

# 预测输出的函数
def predict(network,x):
    W1,W2,W3 = network['W1'],network['W2'],network['W3']
    b1,b2,b3 = network['b1'],network['b2'],network['b3']
    
    #第一层推导
    A1 = np.dot(x,W1)+b1
    Z1 = sigmoid(A1)
    #第二层推导
    A2 = np.dot(Z1,W2)+b2
    Z2 = sigmoid(A2)
    #输出层
    A3 = np.dot(Z2, W3) + b3
    y =softmax(A3)
    
    return y
  
    

上面有两个地方需要注意的：
- normalize=True：这里就是将输入的图片0-255 转换到0-1，这里就是利用了归一化的概念。
- pickle 可以导入相关参数，这是一个经常使用的，就是对数据进行序列化

下面我们需要利用上面的函数来做最后的精度检测，coding如下：

In [23]:
x_test,y_test = get_data()
network = init_network()
acc = 0.

print(len(x_test))

for i in range(len(x_test)):
    y = predict(network,x_test[i])
    num = np.argmax(y)
    if num==y_test[i]:
        acc+=1
        
print("精度为：",float(acc)/len(x_test))
    

10000
精度为： 0.9352


### 批处理

现在我们来会看一下，上面处理测试集的精度的时候，通过一个for循环，一个一个将测试数据放入预测函数中实现预测的。我们这里就看看能否将一组测试数据放入到预测函数中来实现预测。我们来看看我们网络现在各层的shape.

最后的各个数据形态变换如下：

![](imgs/19.jpg)

从图中的结构来看，如果由输入的是一张图片，就会输出的就是一个具有10个元素的一维数组。
这里我们扩展一下我们的思路，如果我们输入的是100张图片，这样的网络能不能正常的输出呢？继续看下图：
![](imgs/20.jpg)

其实该网络还是可以正常的运行的，只不过最后输出的结果会变换成一个[100,10]的输出。也就是y会将100张输入的图片结果都保存起来。

上面的过程就是数据的批处理，批处理的好处：
- 可以大幅缩短每张图像的处理时间
    - 数组运算
    - 减少读入数据的时间



In [9]:
x,_ = get_data()
network = init_network()
W1,W2,W3 = network['W1'],network['W2'],network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [12]:
# 批处理
x,y = get_data()
network = init_network()
# 设置批处理的数量
batch_size = 100
acc = 0.
# 使用range，获得批处理的数量
for i in range(0,len(x),batch_size):
    # 获得当前处理的数据
    x_batch = x[i:i+batch_size]
    y_batch = predict(network,x_batch)
    # 获得当前批次预测的结果数据
    num = np.argmax(y_batch,axis =1)
    # 判断预测值与真实值的是否相等并累加
    acc += np.sum(num == y[i:i+batch_size])

print("精度为：",float(acc)/len(x_test))        

精度为： 0.9352


上面函数使用的几个函数说明  
**range**
 range() 函数可创建一个整数列表，一般用在 for 循环中。
语法如下：range(start, stop[, step])  

>start: 计数从 start 开始。默认是从 0 开始。例如range（5）等价于range（0， 5）;
stop: 计数到 stop 结束，但不包括 stop。例如：range（0， 5） 是[0, 1, 2, 3, 4]没有5
step：步长，默认为1。例如：range（0， 5） 等价于 range(0, 5, 1)

In [7]:
a = range(10)
print(list(a))
print(list(range(5,10)))
print(list(range(1,10,2)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[5, 6, 7, 8, 9]
[1, 3, 5, 7, 9]


**np.argmax**
argmax返回的是最大数的索引.argmax有一个参数axis,默认是0,表示第几维的最大值

>a : array_like 数组
axis : int, 可选默认情况下，索引的是平铺的数组，否则沿指定的轴。
out : array, 可选如果提供，结果以合适的形状和类型被插入到此数组中。
Returns: index_array : ndarray of ints索引数组。
它具有与a.shape相同的形状，其中axis被移除。



In [12]:
import numpy as np
a = np.array([[0, 1, 2],
       [3, 4, 5]])

print(np.argmax(a))
print(np.argmax(a, axis=0))#0代表列
print(np.argmax(a, axis=1))#1代表行

5
[1 1 1]
[2 2]


**numpy 数组比较运算符**

==：比较数组之间的元素是否相等,返回True/False

In [14]:
p = np.array([1,2,3,4])
y = np.array([1,3,2,4])
print(p==y)
print(np.sum(p==y))

[ True False False  True]
2
